<a href="https://colab.research.google.com/github/zaki-moh/MMA_fight_predictor/blob/main/mma_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from google.colab import files
uploaded = files.upload()

Saving large_dataset.csv to large_dataset (3).csv
Saving ufc-fighters-statistics.csv to ufc-fighters-statistics (5).csv


In [78]:
import pandas as pd
stats = pd.read_csv("ufc-fighters-statistics.csv")
historic_fights = pd.read_csv("large_dataset.csv")
# stats.head()
# stats.isnull().sum()
# print(stats.head(15))
# print(historic_fights.head(10))
#print(stats.columns)
stats.columns.tolist()

#

['name',
 'nickname',
 'wins',
 'losses',
 'draws',
 'height_cm',
 'weight_in_kg',
 'reach_in_cm',
 'stance',
 'date_of_birth',
 'significant_strikes_landed_per_minute',
 'significant_striking_accuracy',
 'significant_strikes_absorbed_per_minute',
 'significant_strike_defence',
 'average_takedowns_landed_per_15_minutes',
 'takedown_accuracy',
 'takedown_defense',
 'average_submissions_attempted_per_15_minutes']

In [79]:
# stats.info()
# stats.describe()
# stats.shape
# print(stats.columns.tolist())

In [80]:
historic_fights = historic_fights.iloc[50:]
print(historic_fights)
historic_fights = historic_fights[['r_fighter', 'b_fighter', 'winner']].copy()
historic_fights['winner'] = historic_fights.apply(
    lambda row: row['r_fighter'] if row['winner'].lower() == 'red' else row['b_fighter'],
    axis = 1
)

stats['height_cm'] = stats['height_cm'].fillna(stats['height_cm'].median())
stats['date_of_birth'] = pd.to_datetime(stats['date_of_birth'], errors='coerce')
today = pd.to_datetime('today')
stats['age'] = (today - stats['date_of_birth']).dt.days // 365
stats['age'] = stats['age'].fillna(stats['age'].median())
mean_strikes = stats['significant_strikes_landed_per_minute'].mean()
stats['normalized_strikes'] = stats['significant_strikes_landed_per_minute'] / mean_strikes
stats['performance'] = (0.3 * stats['normalized_strikes']
                        + 0.3 * stats['significant_strike_defence'] / 100
                        + 0.2 * stats['takedown_accuracy'] / 100
                        + 0.2 * stats['takedown_defense'] / 100)
stats['strike_efficiency'] = (
    stats['significant_strikes_landed_per_minute']
    - stats['significant_strikes_absorbed_per_minute']
)
stats['grapple_efficiency'] = (
    0.5 * stats['takedown_accuracy'] / 100 + 0.5 * stats['takedown_defense'] / 100
)
stats['win_indicator'] = (stats['wins'] > stats['losses']).astype(int)
stats['win_ratio'] = stats['wins'] / (stats['wins'] + stats["losses"] + stats['draws'])
stats['win_ratio'] = stats['win_ratio'].fillna(0)

merged = historic_fights.merge(
    stats.add_prefix('r_'),
    left_on='r_fighter',
    right_on='r_name'
)
merged = merged.merge(
    stats.add_prefix('b_'),
    left_on='b_fighter',
    right_on='b_name'
)

merged['height_diff'] = merged['r_height_cm'] - merged['b_height_cm']
merged['reach_diff'] = merged['r_reach_in_cm'] - merged['b_reach_in_cm']
merged['age_diff'] = merged['r_age'] - merged['b_age']
merged['strike_eff_diff'] = merged['r_strike_efficiency'] - merged['b_strike_efficiency']
merged['grapple_eff_diff'] = merged['r_grapple_efficiency'] - merged['b_grapple_efficiency']
merged['performance_diff'] = merged['r_performance'] - merged['b_performance']
merged['win_ratio_diff'] = merged['r_win_ratio'] - merged['b_win_ratio']
merged['r_fighter_win'] = (merged['winner'] == merged['r_fighter']).astype(int)
merged = merged[['r_fighter', 'b_fighter', 'height_diff', 'reach_diff', 'age_diff', 'strike_eff_diff', 'grapple_eff_diff', 'performance_diff', 'win_ratio_diff', 'r_fighter_win']].copy()
print(merged.columns.tolist())
merged.head()

                                event_name         r_fighter  \
50    UFC Fight Night: Moreno vs. Royval 2    Brandon Moreno   
51    UFC Fight Night: Moreno vs. Royval 2    Yair Rodriguez   
52    UFC Fight Night: Moreno vs. Royval 2  Daniel Zellhuber   
53    UFC Fight Night: Moreno vs. Royval 2   Yazmin Jauregui   
54    UFC Fight Night: Moreno vs. Royval 2     Manuel Torres   
...                                    ...               ...   
7434                     UFC 2: No Way Out      Orlando Wiet   
7435                     UFC 2: No Way Out     Frank Hamaker   
7436                     UFC 2: No Way Out     Johnny Rhodes   
7437                     UFC 2: No Way Out     Patrick Smith   
7438                     UFC 2: No Way Out      Scott Morris   

             b_fighter winner         weight_class  is_title_bout gender  \
50      Brandon Royval   Blue            Flyweight              0    Men   
51        Brian Ortega   Blue        Featherweight              0    Men   
52 

,r_fighter,b_fighter,height_diff,reach_diff,age_diff,strike_eff_diff,grapple_eff_diff,performance_diff,win_ratio_diff,r_fighter_win
0,Brandon Moreno,Brandon Royval,-5.08,5.08,-2.0,-0.42,-0.150,0.001077,-0.014286,0
1,Yair Rodriguez,Brian Ortega,7.62,5.08,-1.0,3.03,0.040,0.076153,-0.033333,0
2,Daniel Zellhuber,Francisco Prado,7.62,20.32,3.0,0.84,0.510,0.446537,0.010256,1
3,Yazmin Jauregui,Sam Hughes,-5.08,0.00,-7.0,1.83,0.010,0.307998,0.293706,1
4,Manuel Torres,Chris Duncan,0.00,5.08,-2.0,4.56,-0.195,0.506689,-0.041667,1


In [81]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_cols = ['height_diff',	'reach_diff',	'age_diff',	'strike_eff_diff',	'grapple_eff_diff',	'performance_diff',	'win_ratio_diff',	'win_ratio_diff']

merged[numeric_cols] = scaler.fit_transform(merged[numeric_cols])

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = merged[['height_diff',	'reach_diff',	'age_diff',	'strike_eff_diff',	'grapple_eff_diff',	'performance_diff',	'win_ratio_diff']]
y = merged['r_fighter_win']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7050865800865801
              precision    recall  f1-score   support

           0       0.60      0.43      0.50       636
           1       0.74      0.85      0.79      1212

    accuracy                           0.71      1848
   macro avg       0.67      0.64      0.65      1848
weighted avg       0.69      0.71      0.69      1848



In [83]:
def predictWinner(fighter_A_Name, fighter_B_Name):
  fighter_A = stats.loc[stats['name'] == fighter_A_Name].squeeze()
  fighter_B = stats.loc[stats['name'] == fighter_B_Name].squeeze()

  if fighter_A.empty or fighter_B.empty:
    print("One or both fighters not found in the dataset.")
    return

  features = ['strike_efficiency', 'grapple_efficiency', 'height_cm', 'age', 'performance']

  feature_A = fighter_A[features].values.reshape(1, -1)
  feature_B = fighter_B[features].values.reshape(1, -1)

  A_prob = model.predict_proba(feature_A)[0][1]
  B_prob = model.predict_proba(feature_B)[0][1]

  print(f"{fighter_A_Name} vs {fighter_B_Name}")
  print(f"{fighter_A_Name} likelyhood to win {A_prob:.2%}")
  print(f"{fighter_B_Name} likelyhood to win {A_prob:.2%}")

  if A_prob > B_prob:
    print(f"Predicted winner {fighter_A_Name}")
  else:
    print(f"Predicted winner {fighter_B_Name}")




In [84]:
#Test
predictWinner("Khabib Nurmagomedov", "Conor McGregor")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 5 features, but RandomForestClassifier is expecting 7 features as input.